In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

CONNECT_INDY = False
CONNECT_MOBILE = False
USE_SDK = True

IP_CUR = "192.168.0.10"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

Current PC IP: 192.168.0.10
Mobile ROB IP: 192.168.0.102


In [2]:
INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001
WS_HEIGHT = 1.6
COL_COLOR = (1,1,1,0.2)

from pkg.controller.combined_robot import *
from pkg.project_config import *

if not CONNECT_INDY:
    indy_7dof_client.kiro_tool.OFFLINE_MODE = True
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.indy7, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True} if not USE_SDK else {})

ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[True, CONNECT_INDY])

connection command:
kmb0: True
indy1: False


### Make scene

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.planning.scene import PlanningScene

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)

gscene.set_workspace_boundary(
    -1, 3, -2, 2, -CLEARANCE, WS_HEIGHT, thickness=WALL_THICKNESS)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


### Planning scene

In [4]:
from pkg.geometry.geometry import *
from pkg.utils.code_scraps import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

In [5]:
pscene = PlanningScene(gscene, combined_robot=crob)
ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
CAM_LINK = TIP_LINK.replace("tcp", "link6")
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE
BRUSH_NAME = "brush_face"
brush_face = add_indy_sweep_tool(gscene, ROBOT_NAME, face_name=BRUSH_NAME, tool_offset=0.12, color_col=(0.8,0.8,0.8,0.2))
brush_b = pscene.create_binder(BRUSH_NAME, BRUSH_NAME, WayFramer, point=(0,0,brush_face.dims[2]/2), rpy=(0,np.pi,0))
# from demo_utils.environment import add_kiro_indytool_down
# add_kiro_indytool_down(gscene, zoff=0, tool_link="indy1_tcp", face_name="brush_face", ext_off=0.032, tool_dim=(0.08, 0.32))
gscene.show_pose(crob.home_pose)

HOLD_NAME = "holder"
HOLD_HEIGHT = 0.4
HOLD_KEY = 1
front_dist = 0.45+CLEARANCE
holder_g = gscene.create_safe(GEOTYPE.SPHERE, HOLD_NAME, link_name=HOLD_LINK,dims=(0.01,0.01,0.01),
                           center=(front_dist, 0, HOLD_HEIGHT), rpy=(0,np.pi/2,0),
                           fixed=True, collision=False, color=(1,0,0,0.5))
holder_b = pscene.create_binder(HOLD_NAME, HOLD_NAME, FramedTool, point=(0,0,0), rpy=(0,0,0), key=HOLD_KEY)
floor_b = pscene.create_binder("floor_ws", "floor_ws", PlaceFrame, point=(0,0,WALL_THICKNESS/2))

## Table

In [6]:
TABLE_DIM = (0.6, 1.2, 0.7)
TABLE_NAME = "table"
TABLE_LOC = (2, 0, TABLE_DIM[2]/2)
TABLE_RPY = (0,0,0)
TABLE_KEY = 10

In [7]:
table_g = gscene.create_safe(GEOTYPE.BOX, TABLE_NAME, link_name="base_link", dims=TABLE_DIM,
                             center=TABLE_LOC, rpy=TABLE_RPY,
                             fixed=True, collision=True, color=(0.8,0.8,0.8,1))
table_b = pscene.create_binder(TABLE_NAME, TABLE_NAME, PlaceFrame, point=(0,0,TABLE_DIM[2]/2), key=TABLE_KEY)

## Waypoints

In [8]:
WP_NAME = "wp_{}"
WP_DIM = (0.1,0.1,0.01)
WP_LOC = (-0.2, 0.0, TABLE_DIM[2]/2+WP_DIM[2]/2)
WP_RPY = (0,0,0)

In [9]:
wp_g = gscene.create_safe(GEOTYPE.BOX, WP_NAME.format(0), link_name="base_link", dims=WP_DIM,
                          center=WP_LOC, rpy=WP_RPY,
                          fixed=True, collision=True, color=(0.0,0.0,0.8,1), parent=TABLE_NAME)
waypoint_s = pscene.create_subject(oname="waypoints", gname=TABLE_NAME, _type=WaypointTask,
                                 action_points_dict={
                                     wp_g.name: WayFrame(
                                         wp_g.name, wp_g, [0, 0, wp_g.dims[2] / 2], [0, 0, 0])})

## Bag

In [10]:
BAG_NAME = "bag"
BAG_DIM = (0.4, 0.29,0.635)

In [11]:
bag_g = gscene.create_safe(GEOTYPE.BOX, BAG_NAME, link_name="base_link",dims=BAG_DIM,
                           center=(1,0,BAG_DIM[2]/2), rpy=(0,0,0),
                           fixed=False, collision=True, color=(0.8,0.8,0.8,1))
handle = gscene.create_safe(GEOTYPE.BOX, BAG_NAME+"_hdl", link_name="base_link",dims=(0.2,0.03,0.02),
                           center=(0,0,BAG_DIM[2]/2+0.01), rpy=(0,0,0),
                           fixed=False, collision=True, color=(0.6,0.6,0.6,1), parent=BAG_NAME)

push_point_list = []
for i in range(4):
    R = Rot_axis_series([3, 2], [np.pi/2*i, np.pi/2])
    point = np.round(np.multiply(BAG_DIM, -R[:,2])/2, 4)
    point[2] = -BAG_DIM[2]/2 + HOLD_HEIGHT
    ap_name = "side{}".format(i)
    push_point_list.append(FramePoint(ap_name, bag_g, point=point, rpy=Rot2rpy(R), key=HOLD_KEY))

slide_point = SlidePoint("bottom_p", bag_g, point=(0,0,-BAG_DIM[2]/2-CLEARANCE), rpy=(0,0,0), 
                                           binded_on=floor_b, dist_push=1.0)

bag_s = pscene.create_subject(oname=BAG_NAME, gname=BAG_NAME, _type=PushObject, 
                              push_point_list=push_point_list, slide_point=slide_point)

### prepare planner

In [25]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=100, allow_joint_motion=True)
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
mplan.motion_filters = [gcheck, rcheck]
mplan.incremental_constraint_motion = True

ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

[WARN] reach data is not ready for kmb. Ignoring this robot...


### Custom rules

In [26]:
# joint motion is not allowed when pushing object with holder
idc_push = np.where([stype == PushObject for stype in pscene.subject_type_list])[0]
tplan.explicit_rule = lambda pscene, node, leaf: not any([node[idx]==leaf[idx]=="holder" for idx in idc_push])

In [27]:
# generate base pose to reach targets
N_samples = 50
thetas_available = np.array([np.pi/2*i for i in range(4)])

xy_list = floor_b.geometry.get_vertice_radius_from(crob.home_dict)[0][:,:2]
xy_max = np.max(xy_list,axis=0)
xy_min = np.min(xy_list,axis=0)

targets = [pscene.subject_dict[sname] for sname, stype in zip(pscene.subject_name_list, pscene.subject_type_list) if stype == WaypointTask]
wps_dict = {target.oname: [target.action_points_dict[wname] for wname in target.action_points_order] for target in targets}
def gen_reaching_base(state):
    xyt_samples = np.array([np.random.uniform(xy_min, xy_max).tolist()
                            +[np.random.choice(thetas_available)] 
                            for _ in range(N_samples)])

    # check availability for wps
    Twe = np.linalg.inv(brush_b.Toff_lh)
    Tbe_mat = []
    for target in targets:
        i_s = pscene.subject_name_list.index(target.oname)
        ntem = state.node[i_s]
        wps = wps_dict[target.oname][ntem:] # exclude already reached waypoints
        Tbe_arr = []
        for wp in wps:
            Tbe = np.matmul(wp.get_tf_handle(crob.home_pose), Twe)
            Tbe_arr.append(Tbe)
        Tbe_mat.append(Tbe_arr)
        
    # score with more weights on reachable num on same targets
    score_mat = []
    for i_x, xyt in enumerate(xyt_samples):
        Tbr = gscene.get_tf(ROBOT_BASE, [xyt[0], xyt[1], xyt[2]]+[0]*9)
        score_list = []
        for Tbe_arr in Tbe_mat: # iterate targets
            score_t = 0
            for Tbe in Tbe_arr: # iterate waypoints
                Tre = np.matmul(np.linalg.inv(Tbr), Tbe)
                res = rcheck.check_T(ROBOT_NAME, Tre)
                if res:
                    Q = np.copy(state.Q)
                    Q[:3] = xyt
                    res = mplan.validate_trajectory([Q])
                    score_t += res
            score_list.append(score_t)
        score_mat.append(score_list)
    score_mat = np.square(score_mat) # (pose_num, target_num), square to weight more on large counts
    score_list = np.sum(score_mat, axis=-1,dtype=float)
    score_sum = np.sum(score_list)
    if score_sum>0:
        score_list = np.divide(score_list, score_sum)
        i_select = np.random.choice(range(N_samples), p=score_list)
        Q = np.copy(state.Q)
        Q[:3] = xyt_samples[i_select]
        return Q
    return None

tplan.config_gen = gen_reaching_base

### Plan

In [15]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

['bag', 'waypoints']
('floor_ws', 0)


In [28]:
goal_nodes = [('floor_ws',1)]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=100, 
              multiprocess=True, add_homing=True)

Use 18/36 agents
try: 0 - ('floor_ws', 0)->('floor_ws', 1)
Motion Filer Failure: ReachChecker
result: 0 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 0 - ('floor_ws', 0)->('holder', 0)
try: 0 - ('floor_ws', 0)->('floor_ws', 1)
Motion Filer Failure: ReachChecker
error: 0.35
try transition motion
result: 0 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 0 - ('floor_ws', 0)->('floor_ws', 0)
result: 0 - ('floor_ws', 0)->('floor_ws', 0) = success
branching: 0->1 (0.22/100.0 s, steps/err: 2(2.17914581299 ms)/0)
try: 1 - ('floor_ws', 0)->('floor_ws', 1)
Motion Filer Failure: ReachChecker
result: 1 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 1 - ('floor_ws', 0)->('holder', 0)
Motion Filer Failure: GraspChecker
try: 1 - ('floor_ws', 0)->('floor_ws', 0)
result: 1 - ('floor_ws', 0)->('floor_ws', 0) = success
result: 1 - ('floor_ws', 0)->('holder', 0) = fail
branching: 1->2 (0.29/100.0 s, steps/err: 2(2.93707847595 ms)/0)
try: 2 - ('floor_ws', 0)->('floor_ws', 1)
Motion Filer Failure: ReachChec

Motion Filer Failure: ReachChecker
try transition motion
result: 2 - ('floor_ws', 0)->('holder', 0) = fail
Motion Filer Failure: ReachChecker
result: 0 - ('floor_ws', 0)->('holder', 0) = success
branching: 0->12 (0.92/100.0 s, steps/err: 33(472.8910923 ms)/0.0012625818532)
try: 12 - ('holder', 0)->('floor_ws', 0)
result: 6 - ('floor_ws', 0)->('floor_ws', 1) = fail
result: 11 - ('floor_ws', 0)->('floor_ws', 1) = fail
transition motion tried: False
try: 4 - ('floor_ws', 0)->('floor_ws', 1)
transition motion tried: True
try: 6 - ('floor_ws', 0)->('floor_ws', 1)
try: 5 - ('floor_ws', 0)->('floor_ws', 1)
error: 0.48
transition motion tried: False
try: 3 - ('floor_ws', 0)->('holder', 0)
Motion Filer Failure: ReachChecker
transition motion tried: True
result: 1 - ('floor_ws', 0)->('holder', 0) = success
Motion Plan Failure
Motion Filer Failure: ReachChecker
Motion Filer Failure: ReachChecker
Motion Plan Failure
result: 0 - ('floor_ws', 0)->('holder', 0) = fail
error: 2.3
result: 2 - ('floor_w

error: 2.2
try: 13 - ('holder', 0)->('floor_ws', 0)
result: 4 - ('floor_ws', 0)->('holder', 0) = success
error: 0.45
end
try transition motion
Motion Filer Failure: ReachChecker
constrained motion tried: True
result: 23 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 3 - ('floor_ws', 0)->('floor_ws', 0)
branching: 4->24 (1.28/100.0 s, steps/err: 17(117.45095253 ms)/0.000848719612414)
try: 0 - ('floor_ws', 0)->('holder', 0)
try: 7 - ('floor_ws', 0)->('holder', 0)
Motion Filer Failure: ReachChecker
result: 3 - ('floor_ws', 0)->('floor_ws', 0) = success
error: 2.3
result: 12 - ('holder', 0)->('floor_ws', 0) = success
branching: 3->26 (1.31/100.0 s, steps/err: 2(8.07094573975 ms)/0)
error: 0.64
branching: 12->25 (1.32/100.0 s, steps/err: 46(283.804178238 ms)/1.15326839277e-07)
error: 2.3
result: 5 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 25 - ('floor_ws', 0)->('floor_ws', 1)
try: 26 - ('floor_ws', 0)->('floor_ws', 1)
try transition motion
transition motion tried: True
try transition m

try: 40 - ('floor_ws', 0)->('floor_ws', 1)
try transition motion
end
try: 3 - ('floor_ws', 0)->('floor_ws', 1)
try: 8 - ('floor_ws', 0)->('holder', 0)
try: 35 - ('floor_ws', 0)->('holder', 0)
constrained motion tried: True
Motion Filer Failure: ReachChecker
try: 5 - ('floor_ws', 0)->('floor_ws', 1)
transition motion tried: True
result: 40 - ('floor_ws', 0)->('floor_ws', 1) = fail
error: 2.3
Motion Filer Failure: ReachChecker
try transition motion
Motion Filer Failure: ReachChecker
result: 35 - ('floor_ws', 0)->('holder', 0) = success
branching: 35->42 (1.67/100.0 s, steps/err: 1(6.6339969635 ms)/0)
end
result: 14 - ('holder', 0)->('floor_ws', 0) = success
result: 3 - ('floor_ws', 0)->('floor_ws', 1) = fail
constrained motion tried: True
branching: 14->41 (1.68/100.0 s, steps/err: 77(518.515110016 ms)/1.10016035225e-07)
try: 41 - ('floor_ws', 0)->('floor_ws', 1)
result: 5 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 44 - ('floor_ws', 0)->('floor_ws', 1)
result: 13 - ('holder', 0)->('f

try: 19 - ('floor_ws', 0)->('holder', 0)
try transition motion
try: 23 - ('floor_ws', 0)->('floor_ws', 1)
error: 2.3
try: 29 - ('floor_ws', 0)->('holder', 0)
try: 11 - ('floor_ws', 0)->('holder', 0)
try transition motion
Motion Filer Failure: ReachChecker
result: 23 - ('floor_ws', 0)->('floor_ws', 1) = fail
error: 0.35
end
error: 0.35
try transition motion
constrained motion tried: True
try: 55 - ('floor_ws', 0)->('floor_ws', 1)
result: 14 - ('holder', 0)->('floor_ws', 0) = success
Motion Filer Failure: ReachChecker
try transition motion
branching: 14->56 (2.0/100.0 s, steps/err: 40(297.993898392 ms)/1.09587090947e-07)
result: 55 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 31 - ('floor_ws', 0)->('floor_ws', 1)
try: 56 - ('floor_ws', 0)->('floor_ws', 1)
Motion Filer Failure: ReachChecker
Motion Filer Failure: ReachChecker
result: 31 - ('floor_ws', 0)->('floor_ws', 1) = fail
result: 56 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 35 - ('floor_ws', 0)->('holder', 0)
try: 53 - ('floor

try: 7 - ('floor_ws', 0)->('holder', 0)
try: 51 - ('floor_ws', 0)->('floor_ws', 0)
Motion Filer Failure: GraspChecker
++ adding return motion to acquired answer ++
try: 53 - ('floor_ws', 0)->('floor_ws', 0)
result: 52 - ('floor_ws', 0)->('holder', 0) = fail
try: 51 - ('floor_ws', 0)->('floor_ws', 0)
result: 7 - ('floor_ws', 0)->('holder', 0) = fail
try joint motion
transition motion tried: True
end
result: 51 - ('floor_ws', 0)->('floor_ws', 0) = success
constrained motion tried: True
transition motion tried: False
result: 51 - ('floor_ws', 0)->('floor_ws', 0) = success
branching: 51->72 (2.37/100.0 s, steps/err: 2(3.4761428833 ms)/0)
result: 39 - ('holder', 0)->('floor_ws', 0) = success
Motion Plan Failure
branching: 51->73 (2.38/100.0 s, steps/err: 2(6.54888153076 ms)/0)
branching: 39->74 (2.38/100.0 s, steps/err: 11(81.9478034973 ms)/1.07825406088e-07)
result: 41 - ('floor_ws', 0)->('holder', 0) = success
result: 22 - ('floor_ws', 0)->('holder', 0) = fail
branching: 41->75 (2.39/100.

Goal reached
result: 4 - ('floor_ws', 0)->('holder', 0) = fail
Motion Plan Failure
Motion Filer Failure: ReachChecker
error: 0.37
result: 34 - ('floor_ws', 0)->('holder', 0) = fail
try: 4 - ('floor_ws', 0)->('floor_ws', 1)
try: 86 - ('floor_ws', 0)->('holder', 0)
result: 86 - ('floor_ws', 0)->('floor_ws', 1) = fail
Motion Filer Failure: ReachChecker
transition motion tried: True
try: 27 - ('holder', 0)->('floor_ws', 0)
result: 4 - ('floor_ws', 0)->('floor_ws', 1) = fail
Motion Filer Failure: GraspChecker
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 0)->('holder', 0) = success
result: 27 - ('holder', 0)->('floor_ws', 0) = fail
branching: 0->88 (2.67/100.0 s, steps/err: 3(128.737926483 ms)/0.000624960961709)
result: 86 - ('floor_ws', 0)->('holder', 0) = fail
try: 74 - ('floor_ws', 0)->('floor_ws', 1)
try: 85 - ('floor_ws', 0)->('holder', 0)
try: 0 - ('floor_ws', 0)->('holder', 0)
Motion Filer Failure: GraspChecker
Motion Filer Failure: ReachChecker
result: 85 - ('floor_ws'

branching: 25->102 (3.03/100.0 s, steps/err: 20(523.390054703 ms)/0.00155468018188)
Goal reached
transition motion tried: False
result: 57 - ('floor_ws', 0)->('floor_ws', 0) = success
======================= terminated 9: Stop called from other agent  (3.0/100.0) ===============================
branching: 15->103 (3.04/100.0 s, steps/err: 2(3.23700904846 ms)/0)
branching: 57->105 (3.04/100.0 s, steps/err: 2(3.79800796509 ms)/0)
result: 98 - ('floor_ws', 0)->('floor_ws', 1) = success
======================= terminated 5: Stop called from other agent  (3.0/100.0) ===============================
Motion Plan Failure
result: 80 - ('floor_ws', 0)->('holder', 0) = fail
======================= terminated 17: Stop called from other agent  (3.0/100.0) ===============================
======================= terminated 10: Stop called from other agent  (3.1/100.0) ===============================
======================= terminated 7: Stop called from other agent  (3.1/100.0) =======================

In [29]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

In [30]:
snode_schedule = tplan.get_best_schedule(at_home=True)
ppline.play_schedule(snode_schedule)

('floor_ws', 0)->('floor_ws', 0)
('floor_ws', 0)->('floor_ws', 0)
('floor_ws', 0)->('holder', 0)
('holder', 0)->('floor_ws', 0)
('floor_ws', 0)->('floor_ws', 0)
('floor_ws', 0)->('floor_ws', 1)
('floor_ws', 1)->('floor_ws', 1)
('floor_ws', 1)->('floor_ws', 1)
